In [16]:
import pickle
import config
import function as func
from pymongo import MongoClient
from pprint import pprint
from sqlalchemy.sql import text

import time
from datetime import date, timedelta
import os
import datetime
from dateutil import tz
import pendulum

from db_connect import EngineConnect as DatabaseConnect

In [19]:
class FactDocumentEclaimsModel:
    def __init__(
        self,
        *kwargs,
        request_id: str,
        case_id: str,
        case_number: str,
        import_time_key: int,
        import_date_key: int,
        import_timestamp: str,
        export_timestamp: str,
        export_time_key: int,
        export_date_key: int,
        remark_code: str = None,
    ):
        self.request_id = request_id
        self.case_id = case_id
        self.case_number = case_number
        self.import_time_key = import_time_key
        self.import_date_key = import_date_key
        self.export_time_key = export_time_key
        self.export_date_key = export_date_key
        self.import_timestamp = import_timestamp
        self.export_timestamp = export_timestamp
        self.remark_code = remark_code

class FactProcessEclaimsModel:
    def __init__(
        self,
        *kwargs,
        request_id: str,
        case_id: str,
        case_number: str,
        last_modified_time_key: int,
        last_modified_date_key: int,
        user_name: str = None,
        process_key: int,
        field_name: str,
        field_value: str = None,
        last_modified_timestamp: str
    ):
        self.request_id = request_id
        self.case_id = case_id
        self.case_number = case_number
        self.last_modified_time_key = last_modified_time_key
        self.last_modified_date_key = last_modified_date_key
        self.user_name = user_name
        self.process_key = process_key
        self.field_name = field_name
        self.field_value = field_value
        self.last_modified_timestamp = last_modified_timestamp
        

class EclaimsExcecutor:
    def __init__(
        self,
        *kwargs,
        environment: str,
        uri: str,
        database_name: str,
        docs_collection_name: str, 
        trans_collection_name: str,
        db: DatabaseConnect
    ):
        self.environment = environment
        self.uri = uri
        self.database_name = database_name
        self.docs_collection_name = docs_collection_name
        self.trans_collection_name = trans_collection_name
        self.db = db
        self.start_run = time.time()
        self.maxSevSelDelay = 20000
        self.query = config.ECLAIMS_QUERY
        self.backup_dir = config.BACKUP_DIR
        self.eclaims_docs_dir = config.ECLAIMS_DOCS_DIR
        self.eclaims_trans_dir = config.ECLAIMS_TRANS_DIR
        self.eclaims_backup_dir = config.ECLAIMS_BACKUP_DIR
        self.backup_file_type = config.BACKUP_FILE_TYPE
        self.start = config.start
        self.pre_schema = config.DWH_PRE_DATA_SCHEMA
        self.schema = config.DWH_ANALYTIC_SCHEMA
        self.fact_process_table = config.DWH_FACT_PROCESS_ECLAIMS_TABLE
        self.fact_document_table = config.DWH_FACT_DOCUMENT_ECLAIMS_TABLE
        self.fact_accuracy_table = config.DWH_FACT_ACCURACY_ECLAIMS_TABLE
        self.dim_field_table = config.DWH_DIM_FIELD_ECLAIMS_TABLE
        
    
    def check_connect(self):
        if self.environment == 'development':
            (status, content, time_run) = (True, "good!",  time.time()- self.start_run)
        else:
            client = MongoClient(self.uri, serverSelectionTimeoutMS= self.maxSevSelDelay)
            client.server_info()
            client.close()
            (status, content, time_run) = (True, "good!",  time.time()-self.start_run)
        return {"status": status, "content": content, "time": time_run}
        
    def backup_docs_json(self):
        if self.environment == 'development':
            object_docs = pickle.load(open('./docs.pickle', 'rb'))
            data_docs = [item for item in object_docs]
        else:
            client = MongoClient(self.uri)
            data_query_docs = client[self.database_name][self.docs_collection_name].find(self.query)
            data_docs = [item for item in data_query_docs]
            client.close()
        handle = open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_docs_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type, 'wb')
        pickle.dump(data_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()
            
    def backup_trans_json(self):
        if self.environment == 'development':
            object_trans = pickle.load(open('./trans.pickle', 'rb'))
            data_trans = [item for item in object_trans]
        else:
            client = MongoClient(self.uri)
            data_query_trans = client[self.database_name][self.trans_collection_name].find(self.query)
            data_trans = [item for item in data_query_trans]
            client.close()
        handle = open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_trans_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type , 'wb')
        pickle.dump(data_trans, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()
     
    def pre_fact_process(self):
        key_ignore = ("requestId", "caseId", "caseNumber", "created_date", "last_modified", "documentId", 'attachmentId', 
            'system_processing', 'system_processing123', 'hos_image_type', 'remarkCode', 'remarkDescription')
        key_ignore_trans = ("requestId", "caseId", "caseNumber", "created_date", "last_modified", "documentId", 'attachmentId', 
            'system_processing', 'system_processing123', 'hos_image_type', 'remarkCode', 'remarkDescription', 'Images', 'fileName')
        datas = []
        if self.environment == 'development':
            obj_docs = pickle.load(open('./docs.pickle', 'rb'))
            obj_trans = pickle.load(open('./trans.pickle', 'rb'))
        else: 
            obj_docs = pickle.load(open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_docs_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type, 'rb'))
            obj_trans = pickle.load(open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_trans_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type, 'rb'))
        data_docs = [item for item in obj_docs]
        data_trans = [item for item in obj_trans]
        for data in data_trans:
            records = data['records'][0]
            last_modified = data['last_modified']
            last_modified_utc_7 = last_modified + datetime.timedelta(hours = 7)
            list_keys = list(records.keys())
            list_values = list(records.values())
            user_name = None
            step_type = None
            process_type = 'transform'
            module_type = 'transform_data'
            process_key = func.get_process_key(module_type, process_type, step_type)
            for i in range(len(list_keys)):
                last_modified_date_key_utc_7, last_modified_time_key_utc_7 = func.handle_date_to_date_and_time_id(last_modified_utc_7)
                field_name_temp = list_keys[i]
                if field_name_temp in key_ignore_trans or field_name_temp.startswith('classify'):
                    continue
                elif (field_name_temp.startswith('cl') or field_name_temp.startswith('ocr_')) and field_name_temp != 'claimNature':
                    continue
                field_name = func.lower_first_string(field_name_temp)
                field_value = list_values[i]
                process_object = FactProcessEclaimsModel(
                    case_id = records['caseId'],
                    case_number = records['caseNumber'],
                    request_id = records['requestId'],
                    last_modified_date_key = last_modified_date_key_utc_7,
                    last_modified_time_key = last_modified_time_key_utc_7,
                    last_modified_timestamp = last_modified_utc_7,
                    user_name = user_name,
                    process_key = process_key,
                    field_name = field_name,
                    field_value = field_value
                )
                datas.append(process_object)
        for data in data_docs:
            meta_data = data['project_meta_data']
            last_modified = data['last_modified']
            last_modified_utc_7 = last_modified + datetime.timedelta(hours = 7)
            last_modified_date_key_utc_7, last_modified_time_key_utc_7 = func.handle_date_to_date_and_time_id(last_modified_utc_7)
            if len(data['records']) == 0:
                continue
            records = data['records'][0]
            for key, value in records.items():
                if key == 'keyed_data':
                    for keyed_data in value:
                        if (keyed_data['section'] == 'Auto_Extract' and keyed_data['source'] == 'queue_transform') or \
                        (keyed_data['section'] == 'Verify_Data' and keyed_data['source'] == 'queue_transform'):
                            process_type = func.fix_process_type_keyed_data(keyed_data['section'])
                            user_name = None
                            data_obj = keyed_data['data'][0]
                            list_keys = list(data_obj.keys())
                            list_values = list(data_obj.values())
                            for i in range(len(list_values)):
                                field_name_temp = list_keys[i]
                                step_type = func.fix_step_type_keyed_data(field_name_temp)
                                field_name = func.fix_field_name_keyed_data(field_name_temp)
                                if field_name in key_ignore:
                                    continue
                                field_value = list_values[i]['text']
                                process_key = func.get_process_key(module_type, process_type, step_type)                    
                                process_object = FactProcessEclaimsModel(
                                    case_id = meta_data['caseId'],
                                    case_number = meta_data['caseNumber'],
                                    request_id = meta_data['requestId'],
                                    last_modified_date_key = last_modified_date_key_utc_7,
                                    last_modified_time_key = last_modified_time_key_utc_7,
                                    last_modified_timestamp = last_modified_utc_7,
                                    user_name = user_name,
                                    process_key = process_key,
                                    field_name = field_name,
                                    field_value = field_value
                                )
                                datas.append(process_object)
                elif key == 'system_data':
                    module_type = 'system_data'
                    system_data = value[0]
                    data_obj = system_data['data'][0]
                    auto_qc_output_data = data_obj['auto_qc_output_data']
                    user_name = None
                    process_type = 'automaticQualityControl'
                    step_type = None  
                    process_key = func.get_process_key(module_type, process_type, step_type)
                    if auto_qc_output_data != []:
                        for item in auto_qc_output_data:
                            field_name_temp = item['field_name']
                            field_name = func.lower_first_string(field_name_temp)
                            if field_name in key_ignore:
                                continue
                            process_object = FactProcessEclaimsModel(
                                case_id = meta_data['caseId'],
                                case_number = meta_data['caseNumber'],
                                request_id = meta_data['requestId'],
                                last_modified_date_key = last_modified_date_key_utc_7,
                                last_modified_time_key = last_modified_time_key_utc_7,
                                last_modified_timestamp = last_modified_utc_7,
                                user_name = user_name,
                                process_key = process_key,
                                field_name = field_name,
                                field_value = 1
                            )
                            datas.append(process_object)
                elif key == 'qc_ed_data':
                    module_type = 'qc_ed_data'
                    qc_ed_data = value[0][0]
                    if 'qc_fields_err' not in qc_ed_data.keys():
                        pass
                    else:
                        data_obj = qc_ed_data['qc_fields_err']
                        user_name = qc_ed_data['qcer']
                        step_type = None
                        process_type = func.fix_process_type_keyed_data(qc_ed_data['section'])
                        process_key = func.get_process_key(module_type, process_type, step_type)
                        for item in data_obj:
                            field_name_temp = item['field']
                            if field_name_temp in key_ignore:
                                continue
                            field_name = func.lower_first_string(field_name_temp)
                            field_value = item['value']['text']
                            process_object = FactProcessEclaimsModel(
                                case_id = meta_data['caseId'],
                                case_number = meta_data['caseNumber'],
                                request_id = meta_data['requestId'],
                                last_modified_date_key = last_modified_date_key_utc_7,
                                last_modified_time_key = last_modified_time_key_utc_7,
                                last_modified_timestamp = last_modified_utc_7,
                                user_name = user_name,
                                process_key = process_key,
                                field_name = field_name,
                                field_value = field_value
                            )
                            datas.append(process_object)
                elif key == 'apr_ed_data':
                    module_type = 'apr_ed_data'
                    apr_ed_data = value[0][0]
                    process_type = func.fix_process_type_keyed_data(apr_ed_data['section'])
                    step_type = None
                    process_key = func.get_process_key(module_type, process_type, step_type)
                    data_obj = apr_ed_data['data']
                    for item in data_obj:
                        field_name_temp = item['field']
                        if field_name_temp in key_ignore:
                            continue
                        field_name = func.lower_first_string(field_name_temp)
                        field_value = item['value']['text']
                        user_name = item['aper']
                        process_object = FactProcessEclaimsModel(
                            case_id = meta_data['caseId'],
                            case_number = meta_data['caseNumber'],
                            request_id = meta_data['requestId'],
                            last_modified_date_key = last_modified_date_key_utc_7,
                            last_modified_time_key = last_modified_time_key_utc_7,
                            last_modified_timestamp = last_modified_utc_7,
                            user_name = user_name,
                            process_key = process_key,
                            field_name = field_name,
                            field_value = field_value
                        )
                        datas.append(process_object)
                elif key == 'final_data':
                    module_type = 'final_data'
                    final_data = value[0]
                    user_name = None
                    data_obj = final_data['data']
                    process_type = 'finalize'
                    for item in data_obj:
                        field_name_temp = list(item.keys())[0]
                        if field_name_temp in key_ignore or field_name_temp.startswith('cl') or field_name_temp.startswith('ocr_'):
                            continue
                        step_type = None
                        field_name = func.fix_field_name_keyed_data(field_name_temp)
                        field_value = list(item.values())[0]['text']
                        process_key = func.get_process_key(module_type, process_type, step_type)
                        process_object = FactProcessEclaimsModel(
                            case_id = meta_data['caseId'],
                            case_number = meta_data['caseNumber'],
                            request_id = meta_data['requestId'],
                            last_modified_date_key = last_modified_date_key_utc_7,
                            last_modified_time_key = last_modified_time_key_utc_7,
                            last_modified_timestamp = last_modified_utc_7,
                            user_name = user_name,
                            process_key = process_key,
                            field_name = field_name,
                            field_value = field_value
                        )
                        datas.append(process_object)
        print(datas[0].__dict__)
        self.db.create([item.__dict__ for item in datas], self.pre_schema, self.fact_process_table)
                              
    def fact_document(self):
        datas = []
        if self.environment == 'development':
            obj_docs = pickle.load(open('./docs.pickle', 'rb'))
            obj_trans = pickle.load(open('./trans.pickle', 'rb'))
        else: 
            obj_docs = pickle.load(open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_docs_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type, 'rb'))
            obj_trans = pickle.load(open(self.backup_dir + self.eclaims_backup_dir + self.eclaims_trans_dir + str(self.start.strftime("%Y-%m-%d")) + self.backup_file_type, 'rb'))
        data_docs = [item for item in obj_docs]
        data_trans = [item for item in obj_trans]
        meta_datas = [data['project_meta_data'] for data in data_docs]
        list_created = [data['created_date'] for data in data_docs]
        for data in data_trans:
            if len(data['records']) == 0:
                continue
            records = data['records'][0]
            last_modified = data['last_modified']
            created_date = func.check_index_data_docs(
                meta_datas, list_created, records['requestId'], records['caseId'], records['caseNumber']
            )
            created_date_utc_7 = created_date + datetime.timedelta(hours = 7)
            last_modified_utc_7 = last_modified + datetime.timedelta(hours = 7)
            import_date_key_utc_7, import_time_key_utc_7 = func.handle_date_to_date_and_time_id(created_date_utc_7)
            export_date_key_utc_7, export_time_key_utc_7 = func.handle_date_to_date_and_time_id(last_modified_utc_7)
            remark_code = None
            if records['remarkCode'] != None and records['remarkCode'] != '':
                remark_code = records['remarkCode']
            
            _obj = FactDocumentEclaimsModel(
                case_id = records['caseId'],
                case_number = records['caseNumber'],
                request_id = records['requestId'],
                remark_code = remark_code,
                import_date_key = import_date_key_utc_7,
                import_time_key = import_time_key_utc_7,
                export_date_key = export_date_key_utc_7,
                export_time_key = export_time_key_utc_7,
                import_timestamp = created_date_utc_7,
                export_timestamp = last_modified_utc_7,
            )
            datas.append(_obj)
        print(datas[0].__dict__)
        self.db.create([item.__dict__ for item in datas], self.schema, self.fact_document_table)
            
    def merge_fact_process(self):
        sql_text = \
        """
            -- INSERT OR UPDATE
            WITH ft AS
            (
                SELECT 
                    request_id, case_id, case_number, 
                    last_modified_date_key, last_modified_time_key, 
                    user_name, process_key, field_name, field_value, 
                    last_modified_timestamp,
                    ROW_NUMBER() OVER (PARTITION BY request_id, case_id, case_number, process_key, field_name ORDER BY last_modified_timestamp DESC) AS RN
                FROM {pre_schema}."{table}"
            )
            INSERT INTO {schema}."{table}"
                (request_id, case_id, case_number, last_modified_date_key, last_modified_time_key, user_name, process_key, field_name, field_value, last_modified_timestamp)
            SELECT 
                request_id, case_id, case_number, last_modified_date_key, last_modified_time_key, user_name, process_key, field_name, field_value, last_modified_timestamp
            FROM ft
            WHERE RN = 1
            ON CONFLICT (request_id, case_id, case_number, process_key, field_name)    
            DO
                UPDATE SET
                    last_modified_date_key = EXCLUDED.last_modified_date_key,
                    last_modified_time_key = EXCLUDED.last_modified_time_key,
                    last_modified_timestamp = EXCLUDED.last_modified_timestamp,
                    user_name = EXCLUDED.user_name,
                    field_value = EXCLUDED.field_value;
        """.format(pre_schema = self.pre_schema, table = self.fact_process_table, schema = self.schema)
        self.db.raw_sql(text(sql_text))
        self.db.raw_sql(text("""TRUNCATE TABLE {pre_schema}."{table}";""".format(pre_schema = self.pre_schema, table = self.fact_process_table)))
        
    def fact_accuracy(self):
        sql_1 = """TRUNCATE TABLE {schema}."{table}";""".format(schema = self.schema, table = self.fact_accuracy_table)
        
        sql_2 = """
            INSERT INTO {schema}."{fact_accuracy_table}"
            SELECT DISTINCT
                qca.request_id
                , qca.case_id
                , qca.case_number
                , 5 -- QC
                , qca.field_name
                , true
            FROM {schema}."{fact_process_table}" qca
                JOIN {schema}."{fact_process_table}" qc
                    ON (
                        qca.request_id = qc.request_id AND
                        qca.case_id = qc.case_id AND
                        qca.case_number = qc.case_number AND
                        qca.field_name = qc.field_name AND
                        qca.process_key = 6 AND 
                        qc.process_key = 5 AND
                        replace(trim(qca.field_value), ' ;', ';') <> 
                        replace(trim(qc.field_value), ' ;', ';')
                    );
        """.format(schema = self.schema,  fact_process_table = self.fact_process_table, fact_accuracy_table = self.fact_accuracy_table)
        
        sql_2_1 = """
            INSERT INTO {schema}."{fact_accuracy_table}"
            SELECT DISTINCT
                qca.request_id
                , qca.case_id
                , qca.case_number
                , 3 -- Verify
                , qca.field_name
                , true
            FROM {schema}."{fact_process_table}" qca
                JOIN {schema}."{fact_process_table}" ver
                    ON (
                        qca.request_id = ver.request_id AND
                        qca.case_id = ver.case_id AND
                        qca.case_number = ver.case_number AND
                        qca.field_name = ver.field_name AND
                        qca.process_key = 6 AND 
                        ver.process_key = 3 AND
                        replace(trim(qca.field_value), ' ;', ';') <> 
                        replace(trim(ver.field_value), ' ;', ';')
                    );
        """.format(schema = self.schema,  fact_process_table = self.fact_process_table, fact_accuracy_table = self.fact_accuracy_table)
        
        sql_3 = """
            INSERT INTO {schema}."{fact_accuracy_table}"
            SELECT DISTINCT
                ae.request_id
                , ae.case_id
                , ae.case_number
                , 1 -- Auto Extract
                , ae.field_name
                , true
            FROM {schema}."{fact_process_table}" ae
                JOIN {schema}."{fact_process_table}" f
                    ON (
                        ae.request_id = f.request_id AND
                        ae.case_id = f.case_id AND
                        ae.case_number = f.case_number AND
                        ae.field_name = f.field_name AND
                        ae.process_key = 1 AND 
                        f.process_key = 7 AND
                        replace(trim(ae.field_value), ' ;', ';') <> replace(trim(f.field_value), ' ;', ';')
                    );
        """.format(schema = self.schema,  fact_process_table = self.fact_process_table, fact_accuracy_table = self.fact_accuracy_table)
        
        sql_4 = """
            WITH all_rows AS
            (
                SELECT DISTINCT 
                    rmi.request_id
                    , rmi.case_id
                    , rmi.case_number
                    , rmi.process_key
                    , df.field_name
                FROM {schema}."{fact_process_table}" rmi,
                    {schema}."{dim_field_table}" df
                WHERE
                    rmi.process_key IN (1, 3, 5) AND
                    df.is_sub_field is false
            )
            INSERT INTO {schema}."{fact_accuracy_table}"
            SELECT
                ar.request_id
                , ar.case_id
                , ar.case_number
                , ar.process_key
                , ar.field_name
                , false
            FROM all_rows ar
                LEFT JOIN {schema}."{fact_accuracy_table}" ac
                    ON (
                        ar.request_id = ac.request_id AND
                        ar.case_id = ac.case_id AND
                        ar.case_number = ac.case_number AND
                        ar.field_name = ac.field_name AND
                        ar.process_key = ac.process_key AND
                        ac.field_failed IS true
                    )
            WHERE ac.request_id IS NULL;
        """.format(schema = self.schema,  fact_process_table = self.fact_process_table, fact_accuracy_table = self.fact_accuracy_table, dim_field_table = self.dim_field_table)
        
        sql_5 = """
            INSERT INTO {schema}."{fact_accuracy_table}"
            SELECT DISTINCT
                acae.request_id
                , acae.case_id
                , acae.case_number
                , 4 -- Auto QC
                , acae.field_name
                -- , aqc.field_name
                -- , acae.field_failed
                , CASE
                     WHEN aqc.field_name IS NULL THEN acae.field_failed
                     WHEN aqc.field_name IS NOT NULL THEN NOT acae.field_failed
                  END AS field_failed
            FROM {schema}."{fact_accuracy_table}" acae
                LEFT JOIN {schema}."{fact_process_table}" aqc
                    ON (
                        aqc.request_id = acae.request_id AND
                        aqc.case_id = acae.case_id AND
                        aqc.case_number = acae.case_number AND
                        aqc.field_name = acae.field_name AND
                        aqc.process_key = 4 AND
                        acae.process_key = 1
                    )
            WHERE acae.process_key = 1;
        """.format(schema = self.schema,  fact_process_table = self.fact_process_table, fact_accuracy_table = self.fact_accuracy_table)
        
        sql_6 = """
            UPDATE {schema}."{fact_accuracy_table}" fa
            SET
                exported_timestamp = fd.export_timestamp
            FROM {schema}."{fact_document_table}" fd
            WHERE
                fd.request_id = fa.request_id
                AND fd.case_id = fa.case_id
                AND fd.case_number = fa.case_number
        """.format(schema = self.schema,  fact_accuracy_table = self.fact_accuracy_table, fact_document_table = self.fact_document_table)
        self.db.raw_sql(text(sql_1))
        self.db.raw_sql(text(sql_2))
        self.db.raw_sql(text(sql_2_1))
        self.db.raw_sql(text(sql_3))
        self.db.raw_sql(text(sql_4))
        self.db.raw_sql(text(sql_5))
        self.db.raw_sql(text(sql_6))      
    
    def report(self):
        pass
    
    def clean(self): 
        if self.environment == 'development' or self.environment == 'production':
            now = self.start - timedelta(days=1)
            file_name = str(now.strftime("%Y-%m-%d"))
            docs_file_path = self.backup_dir + self.eclaims_backup_dir + self.eclaims_docs_dir + file_name + self.backup_file_type
            trans_file_path = self.backup_dir + self.eclaims_backup_dir + self.eclaims_trans_dir + file_name + self.backup_file_type
            if os.path.exists(docs_file_path):
                os.remove(docs_file_path)
            else:
                print("The file does not exist")
            if os.path.exists(trans_file_path):
                os.remove(trans_file_path)
            else:
                print("The file does not exist")

In [20]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
eclaims_executor = EclaimsExcecutor(
    environment=config.ENVIRONMENT,
    uri=config.ELROND_URI,
    database_name=config.ELROND_DATABASE,
    docs_collection_name= config.ECLAIMS_DOCS_COLLECTION, 
    trans_collection_name= config.ECLAIMS_TRANS_COLLECTION,
    db = db_connect
)
# eclaims_executor.clean()
# eclaims_executor.check_connect()
# eclaims_executor.backup_trans_json()
# eclaims_executor.fact_document()
eclaims_executor.fact_accuracy()